In [7]:
from ufunctions import *
from functools import reduce
from pandas.tseries.offsets import *


In [2]:
dfQuote = GetQuote(ticker='SAVE', start='2018-01-01')

[*********************100%***********************]  1 of 1 completed


In [3]:
df = dfQuote.copy()
df['Adj Close'] = list(df["Adj Close"].pct_change())
df1 = RelativeStrengthIndex(dfQuote,14)
df2 = macdIndex(dfQuote,12,26,9)

# compile the list of dataframes you want to merge
list_df = [df,df1,df2]
df_final = pd.concat(list_df, axis=1)
df_final.dropna(inplace=True)
df_final.rename(columns={'Close': 'Target'}, inplace=True)
df_final.tail()

,Open,High,Low,Target,Adj Close,Volume,RSI14,SigMACD_12_26_9
Date,,,,,,,,
2021-09-17,24.969999,25.549999,24.610001,25.010000,0.009689,3823300,49.49,-0.499258
2021-09-20,24.629999,25.070000,24.129999,25.059999,0.001999,4735800,49.92,-0.452311
2021-09-21,25.219999,25.540001,24.379999,24.639999,-0.016760,3848100,46.39,-0.412828
2021-09-22,24.780001,25.680000,24.700001,25.260000,0.025162,4030600,51.81,-0.369260
2021-09-23,25.500000,26.620001,25.379999,26.299999,0.041172,4721800,59.25,-0.307982


In [21]:
df_original = df_final.copy()
locShift = 3
df_original.reset_index(inplace=True)
df_original = df_original.append(pd.DataFrame({'Date': pd.date_range(start=df_original.Date.iloc[-1], periods=locShift, freq=BDay(), closed='left')}))
df_original.set_index(['Date'],inplace=True)
df_original.loc[:, df_original.columns != 'Target'] = df_original.loc[:, df_original.columns != 'Target'].shift(locShift)
df_target = df_original[['Target']]
df_features = df_original.loc[:, df_original.columns != 'Target']


df_back_test = pd.merge(df1, df2, left_index=True, right_index=True)

In [23]:
print(df_target.tail())
print(df_features.tail())

               Target
Date                 
2021-09-22  25.260000
2021-09-23  26.299999
2021-09-23        NaN
2021-09-24        NaN
2021-09-27        NaN
                 Open       High        Low  Adj Close     Volume  RSI14  \
Date                                                                       
2021-09-22  24.969999  25.549999  24.610001   0.009689  3823300.0  49.49   
2021-09-23  24.629999  25.070000  24.129999   0.001999  4735800.0  49.92   
2021-09-23  25.219999  25.540001  24.379999  -0.016760  3848100.0  46.39   
2021-09-24  24.780001  25.680000  24.700001   0.025162  4030600.0  51.81   
2021-09-27  25.500000  26.620001  25.379999   0.041172  4721800.0  59.25   

            SigMACD_12_26_9  
Date                         
2021-09-22        -0.499258  
2021-09-23        -0.452311  
2021-09-23        -0.412828  
2021-09-24        -0.369260  
2021-09-27        -0.307982  


In [20]:
df_original.tail()

,Open,High,Low,Target,Adj Close,Volume,RSI14,SigMACD_12_26_9
Date,,,,,,,,
2021-09-22,24.969999,25.549999,24.610001,25.260000,0.009689,3823300.0,49.49,-0.499258
2021-09-23,24.629999,25.070000,24.129999,26.299999,0.001999,4735800.0,49.92,-0.452311
2021-09-23,25.219999,25.540001,24.379999,NaN,-0.016760,3848100.0,46.39,-0.412828
2021-09-24,24.780001,25.680000,24.700001,NaN,0.025162,4030600.0,51.81,-0.369260
2021-09-27,25.500000,26.620001,25.379999,NaN,0.041172,4721800.0,59.25,-0.307982


In [ ]:
# def splitNlag(df):